In [180]:
import tensorflow as tf 
import keras  
from keras.layers import * 
from keras.optimizers import * 
from keras.models import Sequential, Model

In [247]:
def model(image_size = 512, filter_size = 64, full_conv = False, classify=False):
    
    assert image_size % 8 ==0
    model = Sequential() 
    
    model.add(Permute((1,2,3),input_shape = (image_size,image_size,3)))
    #### first layers ########
    model.add(Conv2D(filters=filter_size,kernel_size=(3,3),activation="relu", padding="same",name="conv1_1"))
    model.add(Conv2D(filters=filter_size,kernel_size=(3,3),activation="relu", padding="same",name="conv1_2")) 
    model.add(MaxPool2D(pool_size=2,strides=2, padding="same",name="pool_1"))
    
    #### second layers #######
    model.add(Conv2D(filters=filter_size*2,kernel_size=(3,3),activation="relu", padding="same",name="conv2_1"))
    model.add(Conv2D(filters=filter_size*2,kernel_size=(3,3),activation="relu", padding="same",name="conv2_2")) 
    model.add(MaxPool2D(pool_size=2,strides=2, padding="same",name="pool_2"))
    
    #### third layers #######
    model.add(Conv2D(filters=filter_size*4,kernel_size=(3,3),activation="relu", padding="same",name="conv3_1"))
    model.add(Conv2D(filters=filter_size*4,kernel_size=(3,3),activation="relu", padding="same",name="conv3_2")) 
    model.add(Conv2D(filters=filter_size*4,kernel_size=(3,3),activation="relu", padding="same",name="conv3_3")) 
    model.add(MaxPool2D(pool_size=2,strides=2, padding="same",name="pool_3"))

    #### fourth layers #######
    model.add(Conv2D(filters=filter_size*8,kernel_size=(3,3),activation="relu", padding="same",name="conv4_1"))
    model.add(Conv2D(filters=filter_size*8,kernel_size=(3,3),activation="relu", padding="same",name="conv4_2")) 
    model.add(Conv2D(filters=filter_size*8,kernel_size=(3,3), activation="relu",padding="same",name="conv4_3")) 
    model.add(MaxPool2D(pool_size=2,strides=2, padding="same",name="pool_4"))
    
    #### fifth layers #######
    model.add(Conv2D(filters=filter_size*8,kernel_size=(3,3),activation="relu", padding="same",name="conv5_1"))
    model.add(Conv2D(filters=filter_size*8,kernel_size=(3,3),activation="relu", padding="same",name="conv5_2")) 
    model.add(Conv2D(filters=filter_size*8,kernel_size=(3,3),activation="relu", padding="same",name="conv5_3")) 
    model.add(MaxPool2D(pool_size=2,strides=2, padding="same",name="pool_5"))
    
    if classify: 
        #### flatten image ###### 
        model.add(Flatten(name="flatten"))
        model.add(Dense(4096, activation='relu', name='dense1'))
        model.add(Dense(4096, activation='relu', name='dense2'))
        model.add(Dense(num_classes, activation='softmax', name='predictions'))
    elif full_conv:
        ##### fullt connected layers #########
        model.add(Conv2D(filters=filter_size*64,kernel_size=(1,1),activation="relu", padding="same", name = "fc6"))
        model.add(Conv2D(filters=filter_size*64,kernel_size=(1,1),activation="relu", padding="same", name = "fc7"))
        return model
    return model

In [251]:

def FCN8(model,num_classes=21): 
    
    # Gives the classifications scores for each of the 10 classes including background
    model.add(Convolution2D(num_classes,kernel_size=(1,1),padding="same",activation="relu",name = "score_fc7")) 
    conv_size = model.layers[-1].output_shape[1]
    
    #### bilinear interpolation ##### 
    model.add(Conv2DTranspose(num_classes,kernel_size=4,strides=2,activation=None, padding="valid",name="up_score_fc"))
    extra = model.layers[-1].output_shape[1] - 2*conv_size
    #### crop region added because of valid padding ##### 
    model.add(Cropping2D(cropping=((0,extra),(0,extra)),name="cropping_upscore")) 
    
    ###### skip functiions ###############
    
    ### We add a 1 × 1 convolution layer on top of pool4 to produce additional class predictions. #### 
    pool4 = model.layers[14].output
    cv7 = model.layers[-1].output
    
    fcn_conv1 = Conv2D(num_classes,kernel_size=(1,1), padding="same",activation=None, name = "score_pool4")(pool4)
    add1 = (add(inputs=[fcn_conv1,cv7],name="skip_add_1"))
    deconv1 = Conv2DTranspose(num_classes, padding="valid", kernel_size=(4,4),strides=(2,2),name="score_skip_add_1")(add1)
        
    crop1 = Cropping2D(cropping=((0,2),(0,2)))(deconv1) 
    pool3 = model.layers[11].output
    
    '''
    Decreasing the stride of pooling layers is the most straightforward way to obtain
    finer predictions.
    '''
    final_conv = Conv2D(num_classes, kernel_size=(1,1), padding="same",activation=None,name="fc_final")(pool3)
    add2 = add(inputs=[final_conv,crop1],name="skip_add_2")
    
    
    #Final Up convolution which restores the original image size
    Up = Deconvolution2D(21,kernel_size=(16,16),strides = (8,8),
                         padding = "valid",activation = None,name = "final_score")(add2)
    
    #Cropping the extra part obtained due to transpose convolution
    final = Cropping2D(cropping = ((0,8),(0,8)),name="cropped_final_prediction")(Up)
    
    
    return Model(model.input, final)

In [252]:
vgg16 = model(image_size=512, full_conv=True)
fcn8 = FCN8(model=vgg16)
fcn8.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
permute_111_input (InputLayer)  (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
permute_111 (Permute)           (None, 512, 512, 3)  0           permute_111_input[0][0]          
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, 512, 512, 64) 1792        permute_111[0][0]                
__________________________________________________________________________________________________
conv1_2 (Conv2D)                (None, 512, 512, 64) 36928       conv1_1[0][0]                    
____________________________________________________________________________________________